In [25]:
import pandas_datareader.data as reader
import pandas as pd
import datetime as dt
import statsmodels.api as sm
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
import math
import ast


import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [3]:
path = str(r'C:\Users\tranchinake\Desktop\personale\ScriptPython\momentum')
path = path.replace("\\","/")
os.chdir(path)

In [22]:
#### LETTURA FILE
stocks = pd.read_csv("stocks.csv")
stocks = stocks.set_index('Date')
stocks.index = pd.to_datetime(stocks.index)

fundsret = pd.read_csv("fundsret.csv")
fundsret = fundsret.set_index('Date')
fundsret.index = pd.to_datetime(fundsret.index)
fundsret = fundsret[1:]


factors = pd.read_csv("factors.csv") 
factors = factors.set_index('Date')
factors.index = pd.to_datetime(factors.index)

stocks_pct = stocks.pct_change() ## vengono trasformati in percentuale 
stocks_pct
stocks_pct = stocks_pct[1:]
stocks_pct_monthly = stocks_pct.resample('M').agg(lambda x: (x+1).prod()-1)
stocks_pct_monthly.index = pd.to_datetime(stocks_pct_monthly.index)

fundsret_monthly = fundsret.resample('M').agg(lambda x: (x+1).prod()-1)
fundsret_monthly.index = pd.to_datetime(fundsret_monthly.index)


fundsret3 = factors.join(fundsret)

#fundsret = fundsret[1:]

start = dt.datetime(2019,3,1)
end = dt.datetime(2021,3,31)

# Inizio finestra
startWindow = start
# Fine finestra
endWindow = start + relativedelta(months=+12) 

df = pd.DataFrame()

In [23]:
while(endWindow<=end):
    
    filtroData = (fundsret3.index <= endWindow) & (fundsret3.index >= startWindow)
    filtroStocksData = (stocks_pct.index <= endWindow) & (stocks_pct.index >= startWindow)
    fundsret2 = fundsret3[filtroData]
    stocks_pct2 = stocks_pct[filtroStocksData]
    print("\n")
    print("startWindow : " + str(startWindow))
    print("endWindow : " + str(endWindow))
    print("\n")
    
    # inizio nuova finestra
    startWindow = startWindow + relativedelta(months=+1)
    
    # fine nuova finestra
    endWindow = startWindow + relativedelta(months=+12)
    
    stocks_pct2.index = fundsret2.index
    
    x = fundsret2
    x
    
    y = stocks_pct2
    y
    
#    x = sm.add_constant(x)
#    x
    
    OLS_model = sm.OLS(y,x).fit() # training the model
    predicted_values = OLS_model.predict()  # predicted values
    residual_values = OLS_model.resid # residual values
    coefficient = OLS_model.params # residual values
    coefficient
    residual_values.index = pd.to_datetime(residual_values.index)
    
    residuals = residual_values.resample('12M').agg(lambda x: (x+1).prod()-1) # dà il rendimento mensile
    
    residualMomentum = residuals.iloc[0]
    residualMomentum
    
    residualMomentum = pd.DataFrame(residualMomentum)
    
    residualMomentum['cinquantile'] = pd.qcut(residualMomentum.iloc[:,0],50,labels=False,duplicates='drop')
    
    winners = residualMomentum[residualMomentum['cinquantile'] == 49]
    losers = residualMomentum[residualMomentum['cinquantile'] == 1]
    winners.index
    
    investedMonth = endWindow #+relativedelta(months=+1)
    investedMonth = investedMonth +relativedelta(days=-1)
    investedMonth = pd.to_datetime(investedMonth)
    
    stocks_winners = stocks_pct_monthly[winners.index]
    stocks_winners = stocks_winners[stocks_winners.index == investedMonth]
    stocks_winners['portafoglio'] = stocks_winners.iloc[-1].mean()
    
    stocks_winners.loc[:,'winners'] = str(list(winners.index))
    stocks_winners.loc[:,'winners']
    stocks_winners = stocks_winners[['winners','portafoglio']]
    
    df = df.append(stocks_winners)
    df
#    rendimentoMensile = rendimentoMensile.append(pd.Series(stocks_winners.iloc[-1].mean()), ignore_index=True)



startWindow : 2019-03-01 00:00:00
endWindow : 2020-03-01 00:00:00




startWindow : 2019-04-01 00:00:00
endWindow : 2020-04-01 00:00:00




startWindow : 2019-05-01 00:00:00
endWindow : 2020-05-01 00:00:00




startWindow : 2019-06-01 00:00:00
endWindow : 2020-06-01 00:00:00




startWindow : 2019-07-01 00:00:00
endWindow : 2020-07-01 00:00:00




startWindow : 2019-08-01 00:00:00
endWindow : 2020-08-01 00:00:00




startWindow : 2019-09-01 00:00:00
endWindow : 2020-09-01 00:00:00




startWindow : 2019-10-01 00:00:00
endWindow : 2020-10-01 00:00:00




startWindow : 2019-11-01 00:00:00
endWindow : 2020-11-01 00:00:00




startWindow : 2019-12-01 00:00:00
endWindow : 2020-12-01 00:00:00




startWindow : 2020-01-01 00:00:00
endWindow : 2021-01-01 00:00:00




startWindow : 2020-02-01 00:00:00
endWindow : 2021-02-01 00:00:00




startWindow : 2020-03-01 00:00:00
endWindow : 2021-03-01 00:00:00




In [24]:
df.index = pd.to_datetime(df.index)
df['SPY'] = fundsret_monthly[fundsret_monthly.index.isin(df.index)]

In [27]:
time = np.array(df.index)

In [118]:
fig.data = []

In [150]:
# btc_trace = go.Scatter(x=df.index , y=spy)
# py.iplot([btc_trace])
fig.data = []
fig.layout = {}
fig.add_trace(go.Scatter(
    x=df.index , y=spy,
    name='SPY'
    
))
fig.add_trace(go.Scatter(
    x=df.index , y=portafoglio,
  
    name='PORTAFOGLIO',
    labels=dict(x="Fruit", y="Amount", color="Place")
      
))
# fig.xaxis('werw')
fig.show()

ValueError: Invalid property specified for object of type plotly.graph_objs.Scatter: 'labels'

Did you mean "line"?

    Valid properties:
        cliponaxis
            Determines whether or not markers and text nodes are
            clipped about the subplot axes. To show markers and
            text nodes above axis lines and tick labels, make sure
            to set `xaxis.layer` and `yaxis.layer` to *below
            traces*.
        connectgaps
            Determines whether or not gaps (i.e. {nan} or missing
            values) in the provided data arrays are connected.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets the source reference on Chart Studio Cloud for
            customdata .
        dx
            Sets the x coordinate ste

In [54]:
spy = np.array(df['SPY'])
portafoglio = np.array(df['portafoglio'])

In [53]:
spy

array([-0.12487119,  0.12698358,  0.04764526,  0.01773436,  0.05889221,
        0.06979672, -0.03744358, -0.02493365,  0.10877692,  0.03704831,
       -0.01019039,  0.02780547,  0.04447616])

In [62]:
portafoglio

array([-0.30726008,  0.19174641,  0.13798514,  0.04139223,  0.11232601,
        0.10096079, -0.04936969,  0.00890785,  0.17515035,  0.11516059,
        0.02527946,  0.02036   ,  0.08293518])

In [64]:
df.index

DatetimeIndex(['2020-03-31', '2020-04-30', '2020-05-31', '2020-06-30',
               '2020-07-31', '2020-08-31', '2020-09-30', '2020-10-31',
               '2020-11-30', '2020-12-31', '2021-01-31', '2021-02-28',
               '2021-03-31'],
              dtype='datetime64[ns]', name='Date', freq='M')

In [115]:
from plotly import graph_objs as go
from plotly.subplots import make_subplots


fig = make_subplots(rows=1, cols=2, column_widths=[1,1], print_grid=True, horizontal_spacing=1)
fig.add_trace(go.Scatter(
                x=df.index,y=spy,
                name='SPY',
#                 mode='markers+lines+text',
                text=meta,
                textfont=dict(size=13,color='green'),
                textposition="top right",
                marker_size=8,
                showlegend=True,
                line = go.scatter.Line(color="green", dash='dot')),
                row=1, col=1)
fig.add_trace(go.Scatter(
                x=df.index,y=portafoglio,
                name='portafoglio',
#                 mode='markers+lines+text',
                text=meta,
                textfont=dict(size=13,color='red'),
                textposition="top right",
                marker_size=8,
                showlegend=True,
                line = go.scatter.Line(color="red", dash='dot')),
                row=1, col=1)
fig.show()

This is the format of your plot grid:
[ (1,1) x,y   ]  [ (1,2) x2,y2 ]



In [39]:
df['SPY']

Date
2020-03-31   -0.124871
2020-04-30    0.126984
2020-05-31    0.047645
2020-06-30    0.017734
2020-07-31    0.058892
2020-08-31    0.069797
2020-09-30   -0.037444
2020-10-31   -0.024934
2020-11-30    0.108777
2020-12-31    0.037048
2021-01-31   -0.010190
2021-02-28    0.027805
2021-03-31    0.044476
Freq: M, Name: SPY, dtype: float64